In [4]:
import formulas
import storage
import pandas as pd
import os

if __name__ == '__main__':

    website_creator_path = os.getcwd()
    formula_file_path = website_creator_path + os.path.sep + 'formulas.csv'
    order_file_dir = website_creator_path + os.path.sep + 'order_files'
    docs_dir = storage.get_docs_path(website_creator_path)
    df_formulas =  storage.get_formulas(formula_file_path)
    sort_orders = storage.get_sort_orders(order_file_dir)
    
    storage.delete_directory_if_it_exists(docs_dir)

    #Create formula by year summary
    formula_by_year_summary_df = formulas.get_formula_by_year_summary_df(df_formulas)
    storage.create_sub_directories(docs_dir, formula_by_year_summary_df)
    storage.create_index_files(base_dir=docs_dir, book_collapse=True, sort_orders=sort_orders)
    # formulas.create_formula_files(docs_dir, df_formulas)

    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(df_calculus, formulas.formulas_on_formula_sheet(df_formulas))
    # display(styler_calculus)

docs


IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
sort_orders[sort_orders == 'VIC'].index[0]

6

In [6]:
pd.Series().empty

/tmp/ipykernel_5016/3997566521.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series().empty


True